# Open inTRACKtive WITHIN a Jupyter notebook

### Dependencies

In [ ]:
%env ANYWIDGET_HMR=1
import pandas as pd

from pathlib import Path
from IPython.display import clear_output
from intracktive.notebook_widget import Widget
from intracktive.convert import dataframe_to_browser, make_track_ids_consecutive

### Load data

In [ ]:
# df = pd.read_csv("https://public.czbiohub.org/royerlab/zoo/C_elegans/tracks.csv")
df = pd.read_csv(
    "https://public.czbiohub.org/royerlab/ultrack/inTRACKtive/2D_multicolor_tracks_with_hex.csv"
)
df = make_track_ids_consecutive(df)
df

### Open inTRACKtive widget

In [ ]:
zarr_data_url = dataframe_to_browser(df, Path(), flag_open_browser=False)
clear_output()

widget = Widget(dataset_url=zarr_data_url)
widget

# Get selected tracks

In [ ]:
widget.get_selected_tracks

# Selected specific tracks

In [ ]:
widget.select_tracks([])

# Find longest tracks and select them

In [ ]:
def find_k_longest_tracks(df, k):
    def calculate_start_end_distance(group):
        group = group.sort_values("t")
        start_point = group.iloc[0]
        end_point = group.iloc[-1]
        dx = end_point["x"] - start_point["x"]
        dy = end_point["y"] - start_point["y"]
        return (dx**2 + dy**2) ** 0.5

    # Calculate start-to-end distance for each track
    track_distances = df.groupby("track_id")[["x", "y", "t"]].apply(
        calculate_start_end_distance
    )
    return track_distances.nlargest(k).index.tolist()


longest_tracks = find_k_longest_tracks(df, 5)
widget.select_tracks(longest_tracks)

In [ ]:
def find_longest_track(df):
    track_durations = df.groupby("track_id")["t"].agg(lambda x: x.max() - x.min())
    return track_durations.idxmax()


def find_track_with_most_ancestors(parent_dict):
    def count_ancestors(track_id, visited=None):
        if visited is None:
            visited = set()

        # If we've seen this track before, we have a cycle
        if track_id in visited:
            return 0

        visited.add(track_id)

        # If track has no parent, return 0
        if track_id not in parent_dict:
            return 0

        # Count ancestors of parent + 1 for the parent itself
        return 1 + count_ancestors(parent_dict[track_id], visited)

    # Count ancestors for each track
    ancestor_counts = {
        track_id: count_ancestors(track_id) for track_id in parent_dict.keys()
    }

    # Find track with maximum ancestor count
    return max(ancestor_counts.items(), key=lambda x: x[1])[0]


filtered_df = df[df["parent_track_id"] != -1]
graph = dict(zip(filtered_df["track_id"], filtered_df["parent_track_id"]))

longest_track_id = find_longest_track(df)
print("longest", longest_track_id)

most_ancestors_track_id = find_track_with_most_ancestors(graph)
print("ancestors", most_ancestors_track_id)  # 801